# Search AiiDA Database for Isotherms

In [ ]:
%aiida
import collections
import sys
import ipywidgets as ipw
from IPython.display import clear_output

from aiida.orm import CifData

In [ ]:
def discrete_cmap(colors):
    """Return discrete colormap for matplotlib.
    
    :param: list of RGB colors"""
    N = len(colors)
    base = plt.cm.get_cmap('cubehelix')
    cmap_name = "discrete_" + str(N)
    return base.from_list(cmap_name, colors, N)

def plot(x, y, colors, uuids, ids, title=None, xlabel=None, ylabel=None, clr_label=None):
    import bokeh.plotting as bpl
    import bokeh.models as bmd
    from bokeh.palettes import Viridis256
    from bokeh.io import output_notebook, show
    output_notebook(hide_banner=True)
    
    # to be changed to rest url for Rocio's DB
    show_url = './show.ipynb'

    cmap = bmd.LinearColorMapper(palette=Viridis256, low=min(colors), high=max(colors))
    cbar = bmd.ColorBar(color_mapper=cmap, title=clr_label, location=(0, 0))
    
    hover = bmd.HoverTool(tooltips=[
        ("structure id", "@id"),
    ])
    
    fig = bpl.figure(
        toolbar_location=None,
        title=title,
        x_axis_label=xlabel,
        y_axis_label=ylabel,
        tools=['tap', 'zoom_in', 'zoom_out', 'pan', hover], 
        output_backend='webgl',
    )

    source = bmd.ColumnDataSource(data=dict(x=x, y=y, uuid=uuids, id=ids, color=colors))
    fig.circle('x', 'y', size=10, source=source, fill_color={'field':'color', 'transform':cmap})
    
    if len(x) > 1:
        fig.add_layout(cbar, 'right')
    
    taptool = fig.select(type=bmd.TapTool)
    url="{}?uuid=@uuid".format(show_url)
    taptool.callback = bmd.OpenURL(url=url)
    
    show(fig)

In [ ]:
def search(_=None):
    """Query AiiDA database."""
    
    plot_info.value = "Searching..."
    
    filters = {}

    def add_range_filter(bounds, label):
        filters['attributes.'+label] = {'and':[{'>=':bounds[0]}, {'<':bounds[1]}]}
    
    for k, btn in sliders_dict.items():
        add_range_filter(btn.value, k)
    

    projections = [
        'attributes.'+inp_x.value, 
        'attributes.'+inp_y.value, 
        'attributes.'+inp_clr.value, 
    ]
    

    with output:
        clear_output()
        
        if len(set(projections)) != len(projections):
            plot_info.value = "Please select different quantities for X, Y and Color"
            return

        qb = QueryBuilder()
        qb.append(Str, filters={'attributes.value': molecule.value})
        qb.append(WorkChainNode, project=['uuid'], with_incoming=Str)
        qb.append(Dict, filters=filters, project=projections, with_incoming=WorkChainNode)
        qb.append(CifData, project=['id'], with_outgoing=WorkChainNode)
        results = [k for k in qb.all() if None not in k]
        nresults = len(results)
    
        if nresults == 0:
            plot_info.value = "No results found."
            return

        plot_info.value = "{} results found. Plotting...".format(nresults)

        uuids, x, y, clrs, ids = zip(*results)
        
        plot(
            x=list(map(float, x)),
            y=list(map(float, y)),
            colors=list(map(float, clrs)),
            uuids=list(map(str, uuids)),
            ids=list(map(str, ids)),
            title="{} vs {}".format(inp_y.label, inp_x.label),
            xlabel="{} [{}]".format(inp_x.label, quantities[inp_x.value]['unit']),
            ylabel="{} [{}]".format(inp_y.label, quantities[inp_y.value]['unit']),
            clr_label="{} [{}]".format(inp_clr.label, quantities[inp_clr.value]['unit']))
   
        plot_info.value = "Plotted {} results.".format(nresults)

In [ ]:
# search UI
style = {"description_width":"220px"}
layout = ipw.Layout(width="90%")

quantities = collections.OrderedDict([
    ('Density', dict(label='Density', range=[0.0,20.0], unit='g/cm^3')),
    ('henry_coefficient_average', dict(label='Henry coefficient', range=[0.0,1], unit='mol/kg/Pa')),
    ('adsorption_energy_widom_average', dict(label='Adsorption energy widom', range=[-1e3, 0], unit='kJ/mol')),
    ('POAV_cm^3/g', dict(label='Pore accesible volume', range=[0.0,100.0], unit='cm^3/g')),
    ('PONAV_cm^3/g', dict(label='Pore non-accesible volume', range=[0.0,100.0], unit='cm^3/g')),
    ('Unitcell_volume', dict(label='Pore non-accesible volume', range=[0.0,100000.0], unit='A^3')),
    ('Estimated_saturation_loading', dict(label='Estimated saturation loading', range=[0.0,1000.0], unit='mol/kg')),
])


def get_slider(desc, range, default=None):
    if default is None:
        default = range
    return ipw.FloatRangeSlider(description=desc, min=range[0], max=range[1], 
                                    value=default, step=0.05, layout=layout, style=style)

sliders_dict = collections.OrderedDict()
for k,v in quantities.items():
    desc = "{} [{}]".format(v['label'], v['unit'])
    if not 'default' in v.keys():
        v['default'] = None
        
    slider = get_slider(desc, v['range'], v['default'])
    sliders_dict[k] = slider
    
sliders = list(sliders_dict.values())

In [ ]:
button = ipw.Button(description="Plot")
button.on_click(search)

plot_options = { v['label']: k for k,v in quantities.items() }

inp_x = ipw.Dropdown(
    options = plot_options,
    value = 'Density',
    description='X:',
)

inp_y = ipw.Dropdown(
    options = plot_options,
    value='POAV_cm^3/g',
    description='Y:',
)

inp_clr = ipw.Dropdown(
    options = plot_options,
    value='henry_coefficient_average',
    description='Color:',
)


molecule = ipw.Dropdown(
    options=[
        ("CO2", "co2"),
        ("CH4", "ch4"),
        ("N2",  "n2"),
        ("H2O", "h2o"),
        ("H2",  "h2"),
        ("O2",  "o2"),
    ],
    description="Guest molecule",
    style={'description_width':'initial'},
)

output = ipw.Output()
plot_info = ipw.HTML("")
app = ipw.VBox(sliders + [ipw.HBox([inp_x, inp_y, inp_clr]), ipw.HBox([button, molecule]), output, plot_info])
display(app)